In [2]:
# Import the requisite libraries 
import json 
import spacy 
from spacy.lang.en import English
import sys
import csv

# Load the language 
nlp = spacy.load('en_core_web_lg') 

In [3]:
# A note on index list matching. TEXTS["jobs"][3] tallies with engineering_skills[1]

In [4]:
# Convert Excel to jsonl
# First change the job description to "text" and put in col A
# Then remove all instances of |NA
# Save as csv

In [5]:
#Set path of file to be converted as file_path variable
def convert_csv_jsonl(file_path):
    # Set paths for open file variables
    json_file_name = file_path[0:1] + ".jsonl"
    csvfile = open(file_path, "r")
    jsonfile = open(json_file_name, "w")

    #Set the field names I want to extract
    fieldnames = ("text","County","Title","JobDate","JobUrl")

    #Create the jsonl file
    reader = csv.DictReader(csvfile, fieldnames)
    for row in reader:
        json.dump(row, jsonfile)
        jsonfile.write("\n")
    return jsonfile

In [55]:
convert_csv_jsonl("k.csv")

<_io.TextIOWrapper name='k.jsonl' mode='w' encoding='UTF-8'>

In [6]:
# Create a list for inserting the skills and create stop words
stop_words = ["team","requirement","yesterday","location","bonus","cv","candidate","annum","recruitment","opportunity","benefit","salary","email","e-mail", "-job","hour","work","ref","competent","join","apply","application","starting","role","company","job","applicant","vacancy","client","date","recruiter"]
engineering_skills = []

# Open the JSON with the job advertisements 

with open("k.jsonl") as f: 
    TEXTS = json.loads(f.read()) 

# For every job advertisement, process it using Spacy
i = 0 #Use i to work out how long this might take
import time
start_time = time.clock()
for jt in TEXTS["jobs"]:
    i += 1
    if i % 1000 == 0:
        end_time = time.clock()
        print("The first " + str(i) + " records took " + str(end_time - start_time) + " seconds to process so the projected total completion time is " + str((end_time - start_time)*28.1/60/i*1000) + " minutes") # remove i*1000 if goes wrong
        break #This break allows me to have an idea how long the entire process will take
    doc = nlp(jt["text"])
    # Extract the lemmas of all nouns in the job description and place into the engineering_skills list 
    engineering_skills.append([token.lemma_ for token in doc if not token.is_stop if not token.lemma_ in stop_words if token.pos_ == "NOUN" if token.is_punct == False]) 

The first 1000 records took 64.976293 seconds to process so the projected total completion time is 30.430563888333335 minutes


In [ ]:
Use my jobs skills database to see if these words appear in my ads

In [3]:
# Read the json file with all the skills
import json 
  
# Opening JSON file 
f = open('skills_base.jsonl',) 
  
# returns JSON object as  
# a dictionary 
skills_base = json.load(f) 

In [4]:
# Convert the skills base to a list to ease analysis
list_base = []
for i in skills_base:
    for ii in skills_base[i]:
        list_base.append(ii)  


In [ ]:
# How many matches are there over all jobs in skills_base?

# start_time = time.clock() # Use this if timing it
#Structure for frequency of skills
skills_frequency = {}
counter = 0
for job in engineering_skills:
    for token in job:
        # Is the token in the skills/list base?
        if token in list_base and token is not "c" and token is not "e" is not "b":
            if token not in skills_frequency:
                skills_frequency[token] = 1
            else:
                skills_frequency[token] +=1

In [10]:
# The below containes the frequency list across the first 28100 jobs
skills_frequency
# The below sorts the dictionary in ascending order
print({k: v for k, v in sorted(skills_frequency.items(), key=lambda item: item[1])})

In [6]:
# Extract job title or description by inserting the job no (job) sought

def get_job_title(job): # job should be an integer
    return(TEXTS["jobs"][job]["null"][0])

def get_job_description(job): # job should be an integer
    return(TEXTS["jobs"][job]["text"])

In [12]:
# By job title add all matches in skills_base
    
counter = -1
skillful_list = []
for job in engineering_skills:
    skillful_list.append([get_job_title(counter)])
    counter += 1
    for token in job:
        if token in list_base:
            skillful_list[-1].append(token)


In [14]:
# How many jobs have a match

counter = 0
for i in skillful_list:
    if len(i) > 1:
        counter += 1
print(counter/28100)

0.6949466192170819


In [15]:
# This means 69 per cent return a match

In [7]:
# Convert the engineering skills list to a string so I can compute vectors
def convert_engineering_skills_to_string_for_vectors(eng_skills_list):
# eng_skills_list is e.g. engineering_skills[55] 
    word_string = ""
    i = 0
    for word in eng_skills_list:
        if i>0:
            a = " "
        else:
            a = ""
        word_string = word_string + a + word
        i += 1
    return word_string

In [18]:
# count how many words do not have a vector
# spacy model needs to have been imported as nlp for this to work
def zero_count(word_string):
    vector_test = nlp(word_string)
    zero_count = 0
    for word in vector_test:
        if word.vector[0] == .0:
            zero_count += 1
    return zero_count

In [19]:
# Find which words in list base has the highest match with each word in job description. Minimum match is
# set at similarity_percentage

# Create a holder for all matches of a particular job along with job index number
table_2_list = []

# Create a holder for all matches of a job skill in the skills base
all_matches = []

# Create a job counter to track where we are up to in engineering_skills list
job_counter = 0


# Set the hyperparameter of how similar a word should be, e.g. 90% is 0.9
similarity_percentage = 0.7

while job_counter < len(engineering_skills):

    # for a word in job:
    for word in engineering_skills[job_counter]:

        # Define the engineering job word we are looking at in nlp
        eng_skill_word = nlp(word)

        # If there is no vector in Spacy's model for a word, then ignore it and go to next word
        if eng_skill_word.vector[0] == .0:
            continue

        # Before we loop through the list base, reset the max. skill similarity to be assigned to job
        max_skill_similarity = (0,0,0)

        # for every word in the database of skills
        for list_word in list_base:

            # Define the list base work we are looking at ** remember to increment
            list_base_word = nlp(list_word)

             # If there is no vector in Spacy's model for a word, then ignore it and go to next word
            if list_base_word.vector[0] == .0:
                continue

            # Calculate the similarity between the words    
            word_similarity = (TEXTS["jobs"][job_counter+2]["null"][0],eng_skill_word,eng_skill_word.similarity(list_base_word),list_base_word)

            
            # If this particular comparison is the highest within the list base, then put into max.
            if word_similarity[2] > max_skill_similarity[2]:
                max_skill_similarity = word_similarity
                
            # Alternatively put all matches into a list per job title if higher than the similarity rate define.
            if word_similarity[2] >= similarity_percentage:
                all_matches.append(word_similarity[2])

        # Store or in the case of testing print the max similarity
        if max_skill_similarity[2] >= similarity_percentage:
            print(max_skill_similarity)
            
    # Add the best match to table_2_list
    table_2_list.append([job_counter,max_skill_similarity])
        
    # Go on to the next job
    job_counter += 1
    if job_counter % 100 == 0:
        print(str(test_counter+1) + " jobs processed")

    

('Production Operative', washing, 0.7427796296686087, cleaning)
('Production Operative', cleaning, 1.0, cleaning)
('Production Operative', system, 0.7456541118643616, geographic information system)
('Production Operative', bed, 0.7339403336591056, bed making and linen changes)
('Production Operative', farm, 0.8273069517893382, farm machinery)
('Production Operative', tractor, 0.7519385979626032, agricultural tractors)
('Production Operative', service, 0.816547594972458, claims service)
('Production Operative', contract, 0.8394503041261708, contract management)
('Junior Maintenance Technician', engineering, 0.9107909863617457, mechanical engineering)
('Junior Maintenance Technician', c, 0.7453918601799331, c#)
('Cmc Associate - Pharmaceuticals', product, 0.8621564573982057, product sales)
('Cmc Associate - Pharmaceuticals', customer, 0.8510289638909995, customer contact)
('Cmc Associate - Pharmaceuticals', process, 0.8567770751121945, business process)
('Cmc Associate - Pharmaceuticals'

('Mobile Plant Mechanic/Mobile Plant Fitter', industry, 0.8769697897426163, manufacturing industry experience)
('Dish Potwash', energy, 0.9141725362334587, renewable energy)
('Dish Potwash', efficiency, 0.9004887964809115, energy efficiency)
('Dish Potwash', management, 1.0, management)
('Dish Potwash', energy, 0.9141725362334587, renewable energy)
('Dish Potwash', system, 0.7456541118643616, geographic information system)
('Dish Potwash', management, 1.0, management)
('Dish Potwash', communication, 0.7869038621768647, teaching information and communication technology)
('Dish Potwash', procurement, 1.0, procurement)
('Dish Potwash', supplier, 0.8160913642541081, supplier management)
('Test/Development Engineer - Electronics', maintenance, 0.8392435887500698, vehicle maintenance)
('Test/Development Engineer - Electronics', home, 0.8564608641449741, home building)
('Test/Development Engineer - Electronics', area, 0.7370865925408538, work area maintenance)
('Test/Development Engineer - El

('Project Engineer', customer, 0.8510289638909995, customer contact)
('Project Engineer', employee, 0.8347683491120841, employee relations)
('Project Engineer', experience, 0.8301555915102286, brand experience)
('Project Engineer', contract, 0.8394503041261708, contract management)
('Project Engineer', contact, 0.8473121042355971, customer contact)
('Project Engineer', telephone, 0.7695530837760168, telephone skills)
('Electronic Design Manager Engineering', contract, 0.8394503041261708, contract management)
('Electronic Design Manager Engineering', account, 0.8438351602047116, account closing)
('Electronic Design Manager Engineering', office, 0.8593134868029987, office administration)
('Electronic Design Manager Engineering', service, 0.816547594972458, claims service)
('Electronic Design Manager Engineering', maintenance, 0.8392435887500698, vehicle maintenance)
('Electronic Design Manager Engineering', air, 0.7009296941087901, air travel industry background)
('Electronic Design Mana

('Hgv Technician', language, 0.7263060540002739, teaching speakers of other languages)
('Hgv Technician', engineering, 0.9107909863617457, mechanical engineering)
('Hgv Technician', product, 0.8621564573982057, product sales)
('Hgv Technician', design, 1.0, design)
('Hgv Technician', manufacturing, 0.8690297814033187, manufacturing processes)
('Hgv Technician', process, 0.8567770751121945, business process)
('Hgv Technician', training, 0.8864590760393142, technical training)
('Hgv Technician', development, 0.8897695767548285, community development)
('Hgv Technician', experience, 0.8301555915102286, brand experience)
('Hgv Technician', process, 0.8567770751121945, business process)
('Hgv Technician', closing, 0.7943864044676302, account closing)
('Hgv Technician', maintenance, 0.8392435887500698, vehicle maintenance)
('Drainage Engineer/Service Engineer', facility, 0.8562305558518781, facility supervision)
('Drainage Engineer/Service Engineer', care, 0.8860067570780803, patient care)
('

('Director Of Resident Engagement Strategy', experience, 0.8301555915102286, brand experience)
('Director Of Resident Engagement Strategy', development, 0.8897695767548285, community development)
('Director Of Resident Engagement Strategy', product, 0.8621564573982057, product sales)
('Director Of Resident Engagement Strategy', study, 0.753276077810233, research)
('Director Of Resident Engagement Strategy', closing, 0.7943864044676302, account closing)
('Director Of Resident Engagement Strategy', working, 0.7276716235145376, social work)
('Director Of Resident Engagement Strategy', development, 0.8897695767548285, community development)
('Director Of Resident Engagement Strategy', product, 0.8621564573982057, product sales)
('Director Of Resident Engagement Strategy', product, 0.8621564573982057, product sales)
('Director Of Resident Engagement Strategy', study, 0.753276077810233, research)
('Director Of Resident Engagement Strategy', equipment, 0.854846322535698, equipment cleaning)
(

...

('Mechanical Assembly Operative', advice, 0.7738028578923505, mortgage advice)
('Mechanical Assembly Operative', procurement, 1.0, procurement)
('Mechanical Assembly Operative', strategy, 0.8642919710576753, market strategy)
('Mechanical Assembly Operative', project, 0.8457141128130765, project management)
('Mechanical Assembly Operative', experience, 0.8301555915102286, brand experience)
('Senior Validation Specialist, Vector Substance', bank, 0.775608530330806, bank reconciliation)
('Senior Validation Specialist, Vector Substance', building, 0.8769185623239942, home building)
('Senior Validation Specialist, Vector Substance', consultancy, 0.8781198989239448, business consultancy)
('Senior Validation Specialist, Vector Substance', consultancy, 0.8781198989239448, business consultancy)
('Senior Validation Specialist, Vector Substance', project, 0.8457141128130765, project management)
('Senior Validation Specialist, Vector Substance', service, 0.816547594972458, claims service)
('Senior

('Mechanical Design Engineer - Frontier Pitts', equipment, 0.854846322535698, equipment cleaning)
('Mechanical Design Engineer - Frontier Pitts', facility, 0.8562305558518781, facility supervision)
('Mechanical Design Engineer - Frontier Pitts', product, 0.8621564573982057, product sales)
('Mechanical Design Engineer - Frontier Pitts', system, 0.7456541118643616, geographic information system)
('Mechanical Design Engineer - Frontier Pitts', system, 0.7456541118643616, geographic information system)
('Mechanical Design Engineer - Frontier Pitts', equipment, 0.854846322535698, equipment cleaning)
('Mechanical Design Engineer - Frontier Pitts', risk, 0.8702168370851299, risk management)
('Mechanical Design Engineer - Frontier Pitts', plan, 0.7247812878492718, tax planning)
('Mechanical Design Engineer - Frontier Pitts', equipment, 0.854846322535698, equipment cleaning)
('Mechanical Design Engineer - Frontier Pitts', facility, 0.8562305558518781, facility supervision)
('Mechanical Design E

('Mechanical Design Engineer', process, 0.8567770751121945, business process)
('Mechanical Design Engineer', design, 1.0, design)
('Mechanical Design Engineer', calculation, 1.0, calculation)
('Mechanical Design Engineer', hand, 0.7956689296996887, hand tools)
('Mechanical Design Engineer', experience, 0.8301555915102286, brand experience)
('Mechanical Design Engineer', system, 0.7456541118643616, geographic information system)
('Mechanical Design Engineer', computer, 0.8568531163051184, computer aided)
('Mechanical Design Engineer', literacy, 0.7112095301145296, learning disabilities)
('Mechanical Design Engineer', calculation, 1.0, calculation)
('Mechanical Design Engineer', analysis, 0.8884276499044402, data analysis)
('Mechanical Design Engineer', knowledge, 0.8338488573383724, product knowledge)
('Mechanical Design Engineer', engineering, 0.9107909863617457, mechanical engineering)
('Mechanical Design Engineer', design, 1.0, design)
('Mechanical Design Engineer', process, 0.856777

('Site Surveyor', control, 0.7913155554433292, pollution control)
('Site Surveyor', equipment, 0.854846322535698, equipment cleaning)
('Site Surveyor', employee, 0.8347683491120841, employee relations)
('Site Surveyor', sale, 0.7003441061689079, product sale and delivery)


KeyboardInterrupt: 